In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import geocoder
import json

In [86]:
inspection_data = pd.read_csv("inspections.csv")
inspection_data.head()

,web_certNumber,web_critical,web_customerNumber,web_direct,web_inspectionDate,web_legalName,web_nonCritical,web_reportLink,web_siteName,web_teachableMoments,...,pdf_customer_name,pdf_customer_addr,pdf_certificate,pdf_site_id,pdf_site_name,pdf_insp_type,pdf_date,pdf_animals_total,doccloud_url,licenseType
0,87-R-0002,0,2,0,2015-05-13,UTAH STATE UNIVERSITY,0,https://aphis--c.na107.content.force.com/sfc/d...,UTAH STATE UNIVERSITY,0,...,Utah State University,"V.P. For Research/1450 Old Main Hill\nLogan, U...",87-R-0002,001,UTAH STATE UNIVERSITY,ROUTINE INSPECTION,13-MAY-2015,214.0,https://www.documentcloud.org/documents/234700...,R
1,87-R-0002,0,2,0,2015-05-13,UTAH STATE UNIVERSITY,0,https://aphis--c.na107.content.force.com/sfc/d...,UTAH STATE UNIVERSITY,0,...,Utah State University,"V.P. For Research/1450 Old Main Hill\nLogan, U...",87-R-0002,002,UTAH STATE UNIVERSITY,ROUTINE INSPECTION,13-MAY-2015,0.0,https://www.documentcloud.org/documents/234700...,R
2,87-R-0002,0,2,0,2016-09-26,UTAH STATE UNIVERSITY,1,https://aphis--c.na107.content.force.com/sfc/d...,UTAH STATE UNIVERSITY,2,...,Utah State University,"V.P. For Research/1450 Old Main Hill\nLogan, U...",87-R-0002,001,UTAH STATE UNIVERSITY,ROUTINE INSPECTION,26-SEP-2016,653.0,https://www.documentcloud.org/documents/234700...,R
3,87-R-0002,0,2,0,2016-09-26,UTAH STATE UNIVERSITY,0,https://aphis--c.na107.content.force.com/sfc/d...,UTAH STATE UNIVERSITY,0,...,Utah State University,"V.P. For Research/1450 Old Main Hill\nLogan, U...",87-R-0002,002,UTAH STATE UNIVERSITY,ROUTINE INSPECTION,26-SEP-2016,0.0,https://www.documentcloud.org/documents/234701...,R
4,87-R-0002,0,2,0,2017-09-07,UTAH STATE UNIVERSITY,1,https://aphis--c.na107.content.force.com/sfc/d...,UTAH STATE UNIVERSITY,0,...,Utah State University,"V.P. For Research/1450 Old Main Hill\nLogan, U...",87-R-0002,001,UTAH STATE UNIVERSITY,ROUTINE INSPECTION,07-SEP-2017,594.0,https://www.documentcloud.org/documents/234701...,R


In [87]:
inspection_data.columns = [colName.lstrip('web_') for colName in inspection_data.columns]
inspection_data.columns = [colName.lstrip('pdf_') for colName in inspection_data.columns]

inspection_data.columns

Index(['certNumber', 'critical', 'customerNumber', 'irect', 'inspectionDate',
       'legalName', 'nonCritical', 'reportLink', 'siteName',
       'teachableMoments', 'hash_id', 'iscovered', 'insp_id', 'layout',
       'customer_id', 'customer_name', 'customer_addr', 'certificate',
       'site_id', 'site_name', 'insp_type', 'ate', 'animals_total',
       'occloud_url', 'licenseType'],
      dtype='object')

In [88]:
useful_columns = ['inspectionDate','insp_type','licenseType','critical','nonCritical','teachableMoments','animals_total','legalName','siteName','customer_addr','reportLink']

In [89]:
inspection_data = inspection_data[useful_columns]
inspection_data.dtypes

inspectionDate       object
insp_type            object
licenseType          object
critical              int64
nonCritical           int64
teachableMoments      int64
animals_total       float64
legalName            object
siteName             object
customer_addr        object
reportLink           object
dtype: object

In [90]:
# inspection_data['customer_addr'].str.split('\n', expand=True).sample(5)
def parse_address(df_column):
    # split_address = parsed['Address'].str.split(pat='\n',n=2,expand=True)
    parsed = df_column.str.extract(r'(?P<Street>[\w|\s|,|.]*)\n(?P<City>.*),\s(?P<State>[[A-Z]{2})\s(?P<ZIP>\d\d\d\d\d)',expand=True)
    # split_address = split_address.rename(columns={0:"Address1",1:"Address2",2:"Address3"})
    # parsed = split_address.join(parsed.drop(columns='Address'))
    parsed['Street'] = parsed['Street'].str.replace('\n',', ')

    return parsed

parse_address(inspection_data['customer_addr']).iloc[81671]

/home/gustav/Documents/Programming/Data_Analytics/Animal Welfare Inspections/myenv/lib/python3.10/site-packages/pandas/core/strings/accessor.py:2607: FutureWarning:

Possible nested set at position 52



Street    8520 Allison Pointe Blvd., Suite 400
City                              Indianapolis
State                                       IN
ZIP                                      46250
Name: 81671, dtype: object

In [91]:
len(inspection_data.query('critical > 1 | nonCritical > 8'))

343

In [92]:

multiple_criticals = inspection_data.query('critical > 1 | nonCritical > 8')
multiple_criticals = multiple_criticals.sort_values('animals_total',ascending = False, axis=0)
len(multiple_criticals)

343

In [93]:
# find any entries that have null data and review associated reports

def make_clickable(hyperlink):
    return f'<a target="_blank" href="{hyperlink}">{hyperlink}</a>'.lstrip()

multiple_criticals[multiple_criticals.isna().any(axis=1)].style.format({'reportLink':make_clickable})

KeyboardInterrupt: 

In [ ]:
# The 'City of El Paso' entry has very limited information, so it will be dropped
# The license type can easily be imputed from the report. In this case, they refer to an exotic animal sanctuary that was exhibiting without a license
# Class A refers to a breeder, B to a dealer, R to research, and C to an exhibitor
# Because it helps categorize them, we'll replace license type here with a C

# Reading the report on Nolan and Gina Den Boer reveals that they own dog breeding kennels, which the report indicates contain at least 16 animals, so that value will be updated

try:
    multiple_criticals = multiple_criticals.drop(18013)
except:
    pass
multiple_criticals.at[38578,'animals_total'] = 16.0
multiple_criticals.at[64913,'licenseType'] = 'C'
multiple_criticals.at[40371,'licenseType'] = 'C'
multiple_criticals.at[40372,'licenseType'] = 'C'
multiple_criticals.at[64914,'licenseType'] = 'C'

In [ ]:
# Also because I'm interested in publishing this, we'll replace the license type letters with the type of animal handler

type_dict = {
    'A': 'Breeder',
    'B': 'Dealer',
    'C': 'Exhibitor',
    'R': 'Research',
}

multiple_criticals['type'] = multiple_criticals['licenseType'].map(type_dict)

In [ ]:
#geotag the location using a call to ARCGIS geocoder

multiple_criticals['loc'] = multiple_criticals['customer_addr'].apply(geocoder.arcgis)

In [ ]:
# This is what a location looks like, a JSON object with lots of information enabling data mapping
multiple_criticals['loc'].iloc[20].json

{'address': '1501 S Agnew Ave, Oklahoma City, Oklahoma, 73108',
 'bbox': {'northeast': [35.45137699185795, -97.55402696035198],
  'southwest': [35.44937699185795, -97.55602696035199]},
 'confidence': 9,
 'lat': 35.45037699185795,
 'lng': -97.55502696035198,
 'ok': True,
 'quality': 'PointAddress',
 'raw': {'name': '1501 S Agnew Ave, Oklahoma City, Oklahoma, 73108',
  'extent': {'xmin': -97.55602696035199,
   'ymin': 35.44937699185795,
   'xmax': -97.55402696035198,
   'ymax': 35.45137699185795},
  'feature': {'geometry': {'x': -97.55502696035198, 'y': 35.45037699185795},
   'attributes': {'Score': 98.84, 'Addr_Type': 'PointAddress'}}},
 'score': 98.84,
 'status': 'OK'}

In [ ]:
# See the locations that have no confident geotag and seed if we can retag them using a different method
multiple_criticals[multiple_criticals['loc'].apply(lambda x: x.confidence < 8)]



,inspectionDate,insp_type,licenseType,critical,nonCritical,teachableMoments,animals_total,legalName,siteName,customer_addr,reportLink,type,loc
5881,2022-07-22,ROUTINE INSPECTION,R,2,0,0,2392.0,University of Puerto Rico,SABANA SECA,PO Box 365067 - Medical Science Campus\nSan Ju...,https://aphis--c.na107.content.force.com/sfc/d...,Research,[[00936]]
24517,2015-02-03,ROUTINE INSPECTION,A,2,8,0,1189.0,Oakwood Research Facility Inc,OAKWOOD RESEARCH FACILITY INC,"P.O. Box 455\nOxford, MI 48371",https://aphis--c.na107.content.force.com/sfc/d...,Breeder,"[[48371, Oxford, Michigan]]"
11437,2022-09-15,FOCUSED INSPECTION,B,1,10,0,1026.0,MOUNT HOPE AUCTION INC,"MOUNT HOPE AUCTION, INC.","PO Box 82\nMt Hope, OH 44660",https://aphis--c.na107.content.force.com/sfc/d...,Dealer,"[[44660, Mount Hope, Ohio]]"
5956,2014-02-28,ROUTINE INSPECTION,R,2,2,0,887.0,University of Washington,UNIVERSITY OF WASHINGTON,"Box 357160\nSeattle, WA 98195",https://aphis--c.na107.content.force.com/sfc/d...,Research,"[[98195, Seattle, Washington]]"
5977,2021-01-26,ROUTINE INSPECTION,R,2,1,0,625.0,University of Washington,UNIVERSITY OF WASHINGTON,"BOX 357160\nSEATTLE, WA 98195",https://aphis--c.na107.content.force.com/sfc/d...,Research,"[[98195, Seattle, Washington]]"
12549,2015-01-06,ROUTINE INSPECTION,C,0,23,0,478.0,KARL MOGENSEN,KARL MOGENSEN,"Po Box 88\nNatural Bridge, VA 24578",https://aphis--c.na107.content.force.com/sfc/d...,Exhibitor,"[[24578, Natural Bridge, Virginia]]"
5110,2022-05-11,ROUTINE INSPECTION,R,2,2,0,424.0,NORTH CAROLINA STATE UNIVERSITY,NORTH CAROLINA STATE UNIVERSITY,"CAMPUS BOX 7514\nRALEIGH, NC 27695",https://aphis--c.na107.content.force.com/sfc/d...,Research,"[[27695, Raleigh, North Carolina]]"
8126,2015-01-20,ROUTINE INSPECTION,R,2,2,0,365.0,UNIVERSITY OF OKLAHOMA,RESEARCH BUILDING,"P. O. Box 26901\nOklahoma City, OK 73190",https://aphis--c.na107.content.force.com/sfc/d...,Research,"[[73190, Oklahoma City, Oklahoma]]"
12551,2015-03-10,ROUTINE INSPECTION,C,0,11,0,355.0,KARL MOGENSEN,KARL MOGENSEN,"Po Box 88\nNatural Bridge, VA 24578",https://aphis--c.na107.content.force.com/sfc/d...,Exhibitor,"[[24578, Natural Bridge, Virginia]]"
8505,2019-12-17,ROUTINE INSPECTION,R,2,1,0,280.0,Texas Biomedical Research Institute,SOUTHWEST FOUNDATION FOR BIOMEDICAL RESEARCH,"P. O. Box 760549\nSan Antonio, TX 78245",https://aphis--c.na107.content.force.com/sfc/d...,Research,"[[78245, San Antonio, Texas]]"


In [ ]:
#They're mostly PO Boxes, which can be located to the City, State level at least, but our geocoder should already have done that for us.
#There are a couple of universities in there, but locating them to the ZIP code level should also be fine.
# We'll extract the latitude and longitude data
multiple_criticals['lat'] = multiple_criticals['loc'].apply(lambda x: x.lat)
multiple_criticals['lng'] = multiple_criticals['loc'].apply(lambda x: x.lng)


In [ ]:
# The names of the sites are also bothering me. Some are upper case, some lower case, I'm going to make them all Camel Case for readability

multiple_criticals['siteName'] = multiple_criticals['siteName'].str.upper().str.title()
multiple_criticals['legalName'] = multiple_criticals['legalName'].str.upper().str.title()


In [ ]:
# I should have everything I need to map these locations. I'm going to checkpoint my notebook here so I don't have to keep making API calls
# I'll export the small dataset I'm using as well. The full location information is stored as a ARCGIS object, so it won't export cleanly.
# I really only need the JSON data from it, so I'll extract that, then export the data

multiple_criticals['loc_json'] = multiple_criticals['loc'].apply(lambda x: x.json)
multiple_criticals.to_csv('multiple_criticals.csv') 

In [2]:
# Starting from here when I reload the notebook
multiple_criticals = pd.read_csv('multiple_criticals.csv')


In [3]:
multiple_criticals['type'] = multiple_criticals['type'].astype('category')


In [61]:
test = multiple_criticals.groupby(['lat'])

[i[0] for i in test]

/tmp/ipykernel_74036/2462027868.py:3: FutureWarning:

In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.



[18.417640000000063,
 18.464000000000052,
 25.434923994091776,
 25.614738993167293,
 25.73159097960759,
 26.061239999747485,
 26.26166702074076,
 27.20239486856311,
 27.374507008000023,
 28.118867016452768,
 28.730281004090926,
 28.80874999277701,
 29.39792368998654,
 29.41432160000005,
 29.42381000000006,
 29.43591728897649,
 29.4393909727128,
 29.577221986640808,
 29.651519008749872,
 29.669317016224056,
 29.71652301480832,
 29.727370999626515,
 29.75921099994295,
 29.93901732422094,
 30.12698699260456,
 30.201571011078343,
 30.403978005239736,
 30.44440199641716,
 30.46341970000003,
 30.494034262604103,
 30.824257392643894,
 30.83478901108484,
 30.86954963848484,
 31.00805300800505,
 31.506210000000067,
 31.77039699245728,
 31.827552980663043,
 31.874248018429725,
 32.2545526878344,
 32.38457500000004,
 32.39749800525489,
 32.429900226234615,
 32.53895100324365,
 32.539810000000045,
 32.60091773128008,
 32.68081798343026,
 32.73721500000005,
 32.78793698772226,
 32.89880345992812,
 

In [69]:
# Several reports are for the same location. Ideally I'll display them on the map in a way that prevents overlap
# I want to allow users to access each report individually by clicking

# Add a column that includes the number of reports for each location
multiple_criticals['reportCount']=multiple_criticals.groupby('lat')['lng'].transform('count')

multiple_criticals = multiple_criticals.sort_values('lat')

# Write a function to slightly alter the latitude and longitude for overlapping points, so that they'll show up in a cluster

    
    



def jitter_points(lat,lng, jitter=0.00002):
    """Takes lat, long series and returns lat, long series with slight offset"""
    transformation_matrix = np.array((
        [0,0],
        [1,0],
        [0,1],
        [1,1],
        [0,2],
        [1,2],
        [2,0],
        [2,1],
        [2,2],
        [0,3],
        [1,3],
        [2,3],
    ))

    jitter_lat = []
    jitter_lng = []

    points = {'lat':lat, 'lng':lng}
    points = pd.DataFrame(points)
    grouped = points.groupby('lat')

    for _,frame in grouped:
        for i in range(len(frame)):
            jitter_lat.append(frame['lat'] + transformation_matrix[i][0] * jitter)
            jitter_lng.append(frame['lng'] + transformation_matrix[i][1] * jitter)


    return pd.Series(jitter_lat), pd.Series(jitter_lng)


multiple_criticals['jitter_lat'], multiple_criticals['jitter_lng'] = jitter_points(multiple_criticals['lat'],multiple_criticals['lng'])




    
#AHHHHHH

    

In [70]:
[i for i in multiple_criticals['jitter_lat'][1]]


[18.464000000000052]

In [71]:
multiple_criticals

,Unnamed: 0,inspectionDate,insp_type,licenseType,critical,nonCritical,teachableMoments,animals_total,legalName,siteName,customer_addr,reportLink,type,loc,lat,lng,loc_json,reportCount,jitter_lat,jitter_lng
11,5881,2022-07-22,ROUTINE INSPECTION,R,2,0,0,2392.0,University Of Puerto Rico,Sabana Seca,PO Box 365067 - Medical Science Campus\nSan Ju...,https://aphis--c.na107.content.force.com/sfc/d...,Research,<[OK] Arcgis - Geocode [00936]>,18.417640,-66.076030,"{'address': '00936', 'bbox': {'northeast': [18...",1,"216 28.730301 218 28.730301 Name: lat, d...","216 -81.58715 218 -81.58715 Name: lng, dty..."
132,15140,2017-02-07,ROUTINE INSPECTION,C,0,10,0,143.0,Zoologico De Puerto Rico Dr Juan A Rivero,Zoological De P.R. At Mayaguez,"Po Box 9023207\nSan Juan, PR 00902",https://aphis--c.na107.content.force.com/sfc/d...,Exhibitor,<[OK] Arcgis - Geocode [00902]>,18.464000,-66.112780,"{'address': '00902', 'bbox': {'northeast': [18...",1,"205 36.856436 Name: lat, dtype: float64","205 -76.294784 Name: lng, dtype: float64"
13,5327,2015-09-28,ROUTINE INSPECTION,A,2,1,0,2316.0,The Mannheimer Foundation Inc,Mannheimer Foundation Inc,"20255 S W 360 Street\nHomestead, FL 33034",https://aphis--c.na107.content.force.com/sfc/d...,Breeder,"<[OK] Arcgis - Geocode [20255 SW 360th St, Hom...",25.434924,-80.519220,"{'address': '20255 SW 360th St, Homestead, Flo...",1,"257 29.397924 Name: lat, dtype: float64","257 -82.099775 Name: lng, dtype: float64"
175,28753,2021-07-15,ROUTINE INSPECTION,B,2,1,2,79.0,Zoological Wildlife Foundation Inc,Zoological Wildlife Foundation Inc,"16225 S W 172 Ave\nMiami, FL 33187",https://aphis--c.na107.content.force.com/sfc/d...,Dealer,"<[OK] Arcgis - Geocode [16225 SW 172nd Ave, Mi...",25.614739,-80.470226,"{'address': '16225 SW 172nd Ave, Miami, Florid...",1,"211 39.380375 Name: lat, dtype: float64","211 -97.124705 Name: lng, dtype: float64"
209,10171,2021-06-08,ROUTINE INSPECTION,C,3,3,0,58.0,Festival Fun Parks Llc,Miami Seaquarium,"4400 Richenbacker Causeway\nMiami, FL 33149",https://aphis--c.na107.content.force.com/sfc/d...,Exhibitor,<[OK] Arcgis - Geocode [4400 Rickenbacker Cswy...,25.731591,-80.165475,"{'address': '4400 Rickenbacker Cswy, Key Bisca...",1,37 40.526616 300 40.526616 41 40.52...,37 -83.215946 300 -83.215946 41 -83.21...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,54924,2015-03-20,ROUTINE INSPECTION,C,0,9,0,42.0,Kirsty Lee L L C,"Kirsty Lee, L L C","8311 N.E. 166th St.\nKenmore, WA 98028",https://aphis--c.na107.content.force.com/sfc/d...,Exhibitor,"<[OK] Arcgis - Geocode [8311 NE 166th St, Kenm...",47.748668,-122.229615,"{'address': '8311 NE 166th St, Kenmore, Washin...",3,"312 41.373696 Name: lat, dtype: float64","312 -71.954111 Name: lng, dtype: float64"
153,54928,2015-11-30,ROUTINE INSPECTION,C,0,12,0,103.0,Kirsty Lee L L C,"Kirsty Lee, L L C","8311 N.E. 166th St.\nKenmore, WA 98028",https://aphis--c.na107.content.force.com/sfc/d...,Exhibitor,"<[OK] Arcgis - Geocode [8311 NE 166th St, Kenm...",47.748668,-122.229615,"{'address': '8311 NE 166th St, Kenmore, Washin...",3,34 38.113215 35 38.113215 14 38.11321...,34 -88.767207 35 -88.767207 14 -88.76720...
65,28638,2017-05-01,FOCUSED INSPECTION,R,2,0,0,342.0,Snbl Usa Ltd,"Snbl Usa, Ltd","6605 Merrill Creek Parkway\nEverett, WA 98203",https://aphis--c.na107.content.force.com/sfc/d...,Research,<[OK] Arcgis - Geocode [6605 Merrill Creek Pkw...,47.938321,-122.266999,"{'address': '6605 Merrill Creek Pkwy, Everett,...",2,"220 33.305318 Name: lat, dtype: float64","220 -84.127287 Name: lng, dtype: float64"
17,80270,2021-07-14,ROUTINE INSPECTION,R,4,0,0,2081.0,Altasciences Preclinical Seattle Llc,Altasciences Preclinical Seattle Llc,"6605 Merrill Creek Parkway\nEverett, WA 98203",https://aphis--c.na107.content.force.com/sfc/d...,Research,<[OK] Arcgis - Geocode [6605 Merrill Creek Pkw...,47.938321,-122.266999,"{'address': '6605 Merrill Creek Pkwy, Everett,...",2,75 29.435937 108 29.435937 105 29.43...,75 -95.170922 108 -95.170

In [52]:
# Quick sanity check, are they all in the US? Are they plotted correctly? I'll create the plot structure I'm ultimately going to use to create the final map

# fig1 = go.FigureWidget([go.scattergeo(multiple_criticals, title='Animal Welfare Inspections 2014-2022', lat='lat', lon='lng', scope='north america', color='type', size = 'animals_total', size_max=30, hover_name= "siteName", hover_data=hover_data, fitbounds='locations', projection='natural earth', width=1000, height=800
# )])

fig = go.Figure(data=go.Scattergeo(
    lat=multiple_criticals['jitter_lat'],
    lon=multiple_criticals['jitter_lng'],
    text=multiple_criticals['legalName'],
    mode='markers',
    customdata=[multiple_criticals['critical'], multiple_criticals['nonCritical'], multiple_criticals['type']]
)

)

fig.show()

They're all there! Now to fine-tune the map. 

* The area should include just the United States - the map will be specified as a geo object
* Marker size depends on # of animals total (logarithmic scale)
* Marker color depends on type of animal handler
* There should be some jitter to allow users to access multiple reports from the same site
* On hover, each marker should show the name of the site and the number of critical and noncritical violations
* On click, each marker should open a new tab with the PDF report
* The legend will explain the types of site

In [ ]:
# Original code used when I was trying to do this with plotly express
# labels = {
#     'animals_total': '# of Animals',
#     'type': 'Category of animal handler',
#     'critical': "Critical violations",
#     'nonCritical': "Noncritical violations",

# }

# hover_data = {
#     'animals_total': ':.0f',
#     'critical': True,
#     'nonCritical': True,
#     'type': False,
#     'lat': False,
#     'lng': False,
# }





# fig = px.scatter_geo(multiple_criticals, title='Animal Welfare Inspections 2014-2022', labels=labels, lat='lat', lon='lng', scope='north america', color='type', size = 'animals_total', size_max=30, hover_name= "siteName", hover_data=hover_data, fitbounds='locations', projection='natural earth', custom_data=['reportLink'], width=1000, height=800)
# fig.show()


In [99]:
# Start with the map

title = ("U.S. Animal Welfare Inspections 2014-2022<br>" +
        "<sup>All reports that found 2+ critical issues or 8+ non-critical ones</sup>")
geo = dict(
    scope ='usa',
    projection_type='albers usa',
    showland = True,
    landcolor = 'rgb(230,230,230)',
    subunitwidth = 0.5,
    
)

fig.update_layout(title=title,geo=geo)

In [100]:
# Then the markers

marker = dict(
    size = multiple_criticals['animals_total'],
    sizeref = 30,
    sizemin = 3,
    sizemode = 'area',
    opacity = 0.7,
    symbol = 'circle',
    colorscale = ['red','blue','green','yellow'],
    color = multiple_criticals['type'].cat.codes
)
fig.update_traces(overwrite=True,marker=marker)

In [79]:
fig.data[0]

Scattergeo({
    'hoverinfo': 'text',
    'hovertext': array(['Martha Holmes', 'Elm Hill Labs Inc', 'Santa Cruz Biotechnology Inc.',
                        ..., 'Oklahoma State University - Oklahoma City',
                        'R B H S New Jersey Medical School', 'Nolan Den Boer Gina Den Boer'],
                       dtype=object),
    'lat': array([40.36965901, 42.58394309, 36.954651  , ..., 35.46910899, 40.52574899,
                  43.17552901]),
    'lon': array([ -75.54738698,  -71.38845854, -122.05165397, ...,  -97.58243302,
                   -74.470282  ,  -96.39315604]),
    'marker': {'opacity': 0.7,
               'size': array([15780., 10000.,  6400., ...,     0.,     0.,    16.]),
               'sizemin': 3,
               'sizemode': 'area',
               'sizeref': 30,
               'symbol': 'circle'},
    'mode': 'markers',
    'text': array(['Martha Holmes', 'Elm Hill Labs Inc', 'Santa Cruz Biotechnology Inc.',
                   ..., 'Oklahoma State Universi

In [78]:
# Now the hovertext

hovertext = multiple_criticals['legalName']
hoverinfo = 'text'

fig.update_traces(overwrite=True, hovertext=hovertext, hoverinfo=hoverinfo)

In [80]:
# Change some of the formatting, like the way the mouseover looks

print(fig.data[0].hovertemplate)

None


In [92]:
f2.add_scattergeo(fig)



ValueError: 
    Invalid value of type 'plotly.graph_objs._figure.Figure' received for the 'connectgaps' property of scattergeo
        Received value: Figure({
    'data': [{'customdata': array([['https://aphis--c.na107.content.force.com/sfc/dist/version/download/?oid=00Dt0000000GyZH&ids=068t000000Dnw9Q&d=%2Fa%2F3d000000pdQs%2FBdTeXYiEOMGHsQXQBDzoqIh9Qc2L6WcjNm8Mre5gLww&asPdf=false',
                                    15780.0, 0, ..., 'Dealer', 40.369659010493024, -75.54738698319876],
                                   ['https://aphis--c.na107.content.force.com/sfc/dist/version/download/?oid=00Dt0000000GyZH&ids=068t000000ERYb6&d=%2Fa%2F3d000000pkqz%2F0Xm5lDZkhpTx8FP.LRH45BKoC1Zekv5njEdwxlFr2No&asPdf=false',
                                    3953.0, 1, ..., 'Dealer', 30.834789011084837, -87.14930898842836],
                                   ['https://aphis--c.na107.content.force.com/sfc/dist/version/download/?oid=00Dt0000000GyZH&ids=068t000000awYYz&d=%2Fa%2F3d000000pVEW%2F3DsFMhtMO